In [1]:
# prueba para leer presentaciones y
# enviarlas a OpenAI y recibir una
# respuesta MINERVA

In [2]:
from dotenv import load_dotenv, find_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [15]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

In [16]:
def db_from_info(pdf):
    info = ""
    lector = PdfReader(pdf)
    for page in lector.pages:
        info += page.extract_text()
    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    docs = text_splitter.split_text(info)

    db = FAISS.from_texts(docs, embeddings)
    return db

In [17]:
def get_response(db, query, k=7):
    docs = db.similarity_search(query, k=k)
    docs_content = ' '.join([d.page_content for d in docs])

    chat = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0.7)

    template = """
        Eres Minerva, un asistente muy carismática de investigación profesional que habla en español.
        
        Tu tarea consiste en leer detalladamente la información proporcionada principalmente
        en documentos de formato PDF, para dar respuestas a cualquier tipo de pregunta,
        en especial a preguntas puntuales.

        SIEMPRE debes responder dentro del siguiente contexto proporcionado aqui: {docs}.
        
        Si no sabes la respuesta, debes decir: 'No sé'.
        
        NUNCA debes empezar un párrafo con la frase 'En resumen,'.

        Tus respuestas deben ser precisas y detalladas.
        """
    
    default_prompt = SystemMessagePromptTemplate.from_template(template)

    pregunta_humana = 'Responde la siguiente pregunta: {pregunta}'
    query_humano = HumanMessagePromptTemplate.from_template(pregunta_humana)

    chat_prompt = ChatPromptTemplate.from_messages(
        [default_prompt, query_humano]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    respuesta = chain.run(pregunta=query, docs=docs_content)
    respuesta = respuesta.replace("\n", "")
    return respuesta, docs

In [18]:
# Demo

pdf = "comvaca.pdf"
db = db_from_info(pdf)

In [16]:
#provi.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Cuáles son las cifras de Providencia?
----------------------------------
Respuesta: Las cifras de Providencia para el año 2022 son las siguientes:- EBITDA: COP
412,508,000- Ventas: COP 1,270,000,000- Número total de empleados: 2,928 (138
mujeres, 2,790 hombres)- Contratistas: 681- Caña molida (Toneladas): 3,446,803-
Producción de azúcar (Quintales): 7,360,058- Producción alcohol (Litros): 76,580,108-
Cogeneración de energía (Kilovatio hora): 277,188,664- Producción compostaje
(Toneladas): 75,742.62- Formados en Providencia Centro de Formación Integral: 3,295
personasEstas son las cifras destacadas de la organización Providencia para el año
2022.


In [43]:
#diagnostico.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Qué es AIRBNB Sale?
----------------------------------
Respuesta: AIRBNB Sale es una iniciativa que brinda la posibilidad de ofertar inmuebles
destinados al turismo y asesorar a posibles clientes en futuras inversiones. Esta
plataforma ofrece la oportunidad de invertir en inmuebles en destinos turísticos como
República Dominicana, México, Colombia y Argentina, permitiendo a los inversionistas
ofertar sus propiedades para el turismo.


In [6]:
#diagnostico.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

c:\Users\luisr\OneDrive\Desktop\chatbots\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\luisr\OneDrive\Desktop\chatbots\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Pregunta: ¿Cuál es el tema principal del documento?
----------------------------------
Respuesta: El tema principal del documento es la identificación y análisis de problemas
empresariales en diferentes organizaciones, así como la propuesta de soluciones y
compromisos por parte de los consultores para abordar dichos problemas. Se abordan
aspectos como la calidad del servicio, la satisfacción del cliente, la estructura
financiera, la oferta de cursos de extensión, la implementación de modelos de
negocio, la innovación educativa, entre otros. Cada análisis se realiza con el
objetivo de mejorar el desempeño y la sostenibilidad de las empresas involucradas.


In [8]:
#TG00814.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Cuál es el tema principal?
----------------------------------
Respuesta: El tema principal es la investigación sobre los factores clave de éxito y la
competitividad de las empresas en el Valle del Cauca. Se aborda cómo algunas empresas
en esta región han logrado establecerse como líderes del mercado gracias a su enfoque
en la competitividad y la adopción de estrategias que les permiten diferenciarse y
añadir valor a sus productos y servicios. La investigación se basa en la teoría de
las cinco fuerzas competitivas de Michael Porter y busca comprender cómo tanto los
consumidores como los dirigentes perciben la competitividad en las empresas de la
región.


In [10]:
#TG00814.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Eres capaz de identificar los autores del proyecto de grado?
----------------------------------
Respuesta: No se mencionan los autores del proyecto de grado en la información proporcionada. No
es posible identificar a los autores basándose en el texto proporcionado.


In [12]:
#induccion.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Para qué sirve el horno de convección?
----------------------------------
Respuesta: El horno de convección sirve para hornear y asar alimentos de manera uniforme. Este
tipo de horno utiliza un sistema de ventilación que distribuye el calor de manera
constante alrededor de los alimentos, lo que ayuda a que se cocinen de manera más
rápida y uniforme. Gracias a esta circulación de aire caliente, se logra una cocción
más eficiente y resultados más consistentes en la preparación de alimentos.


In [13]:
#induccion.pdf

query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Qué procedimientos realizan en el restaurante?
----------------------------------
Respuesta: En el restaurante se realizan los siguientes procedimientos:1. **Preparación de
Alimentos:** Se sigue las recetas y pautas de presentación para garantizar la
consistencia en los platos. Esto implica seguir las instrucciones detalladas para
preparar cada plato de acuerdo con los estándares establecidos.2. **Limpieza de la
Cocina:** Se mantiene la estación de trabajo limpia y ordenada en todo momento. Esto
incluye limpiar las superficies de trabajo, los utensilios y equipos utilizados, así
como mantener la cocina en general en óptimas condiciones de higiene.3. **Control de
Inventario:** Se registra los ingredientes utilizados y se notifica al Jefe de
Partida cuando sea necesario reponer existencias. Es importante llevar un registro
preciso de los ingredientes para asegurar que siempre haya suficiente stock
disponible para preparar los platos.Estos procedimientos son fundamentales para


In [19]:
query = input('Hazme una pregunta:')
respuesta, docs = get_response(db, query)
print('Pregunta:', query)
print('----------------------------------')
print('Respuesta:', textwrap.fill(respuesta, width=85))

Pregunta: ¿Cuál es el tema central del documento?
----------------------------------
Respuesta: El tema central del documento es la competitividad, especialmente enfocada en los
aspectos relacionados con la productividad, el desarrollo económico, la
globalización, la innovación, la tecnología, la conectividad y la estrategia
competitiva a nivel nacional, regional y local. Se destaca la importancia de integrar
la productividad con la mejora de la calidad de vida, el respeto por el medio
ambiente, el fomento del emprendimiento y la educación a través de políticas
gubernamentales y sociales de apoyo. La competitividad se presenta como un factor
clave para el crecimiento a mediano y largo plazo, en un contexto marcado por la
globalización y la interconexión económica a nivel mundial.
